# 🚌 Projet MDM - Mobilité Durable en Montagne ⛰️

*Auteur : Thibault Launois*

*Date : 26/10/2025*

**Description :**

Ce script charge les données de **List_iti_D4G_isere_output_altitude.csv** généré par **2025-10-20_TLaunois_C2C_altitude.ipynb** et va créer un geoparquet.
Les points de départ pour chaque topo n'étant pas fourni par C2C, on définit le point de départ comme étant le point d'un topo ayant l'altitude la plus basse.

# Importation des modules

In [ ]:
import geopandas as gpd
import pandas as pd

# Charger les données

In [ ]:
df = pd.read_csv("../data/C2C/Liste_iti_D4G_isere_output_altitude.csv", index_col=0)
df.head(2)

,Titre itinéraire,Id itinéraire,url,activité,durée,dénivelé (m),longueur (m),Région/massif,Id wp principal,"(id WP - titre - [X,Y] - accessibilité (si renseigné) )",X,Y,lon,lat,altitude
0,Les Bans:Spigolo Sud du Contrefort Médian. Voi...,1780070,https://www.camptocamp.org/routes/1780070,mountain_climbing,['1'],NaN,0.0,(14328 - Isère)(14403 - Écrins),39056.0,"(39056 - Les Bans - [705350.3499286951, 55976...",705350.349929,5.597690e+06,6.336270,44.848420,3564.832275
1,Les Bans:Spigolo Sud du Contrefort Médian. Voi...,1780070,https://www.camptocamp.org/routes/1780070,mountain_climbing,['1'],NaN,0.0,(14328 - Isère)(14403 - Écrins),39056.0,(127458 - Entre Les Aygues - [712692.092985492...,712692.092985,5.594929e+06,6.402222,44.830833,1613.258789


# Déterminer le point de départ pour chaque topo

On fait l'hypothèse que le point de départ d'un topo est le point ayant l'altitude la plus basse.\
La colonne "Id Itinéraire" permet de compter le nombre de topo dans la base de donnée, attention différents topos peuvent avoir le même nom.

In [ ]:
# je crée le dataframe avec le point de départ pour chaque topo
data = []

for id in df["Id itinéraire"].unique():
    data.append(list(df[df["Id itinéraire"] == id].sort_values("altitude").iloc[0]))

# aws = access way point
df_awp = pd.DataFrame(data=data, columns=df.columns)

# Rajout d'une colonne avec l'id du point du départ
Dans la colonne **(id WP - titre - [X,Y] - accessibilité (si renseigné) )**, on a les informations du point de départ.

In [87]:
dict_data = dict()
data2 = []

for index, row in df_awp.iterrows():
    # id du point de départ
    id_wp = int(
        row["(id WP - titre - [X,Y] - accessibilité (si renseigné) )"]
        .strip("()")
        .split(" - ")[0]
    )
    data2.append(id_wp)

    # enregistre
    if id_wp not in dict_data:
        name = (
            row["(id WP - titre - [X,Y] - accessibilité (si renseigné) )"]
            .strip("()")
            .split(" - ")[1]
        )
        dict_data[id_wp] = [name, row["lon"], row["lat"]]

df_awp["Id wp"] = data2

In [ ]:
# on regroupe et compte le nombre de topo par point de départ
df_awp2 = df_awp.groupby("Id wp").nunique()["Id itinéraire"]

In [89]:
data_name = []
data_lat = []
data_lon = []
data_nbr_topo = []

for index in df_awp2.index:
    data_name.append(dict_data[index][0])
    data_lon.append(dict_data[index][1])
    data_lat.append(dict_data[index][2])
    data_nbr_topo.append(df_awp2[index])

In [ ]:
df_awp2 = pd.DataFrame(
    {
        "Id wp": df_awp2.index,
        "Name wp": data_name,
        "lat": data_lat,
        "lon": data_lon,
        "nbr_topo": data_nbr_topo,
    }
)

In [ ]:
# df_awp2.to_csv("../data/C2C/Liste_point_depart_isere.csv", index = False)

# Création du geo-parquet
Il faut renommer les colonnes afin de coller au format pour le géoparquet

In [ ]:
# Je renomme les colonnes existantes avec le bon nom
df_geoparquet = df_awp2.rename(
    columns={"Name wp": "name", "Id wp": "navitia_id", "nbr_topo": "other"}
)


# je mets l'info du nombre de topos au bon format
def to_dict(x):
    return {"nombre_de_depart_de_topo": x}


df_geoparquet["other"] = df_geoparquet["other"].map(to_dict)

In [ ]:
# je crée les colonnes inexistantes et je les rempli de vide
df_geoparquet["gtfs_id"] = ""
df_geoparquet["osm_id"] = ""
df_geoparquet["description"] = ""
df_geoparquet["line_gtfs_ids"] = ""
df_geoparquet["line_osm_ids"] = ""
df_geoparquet["geometry"] = ""

In [ ]:
# je transforme la colonne latitude et longitude en points (lat,lon) en créant le geo-dataframe
gdf = gpd.GeoDataFrame(
    df_geoparquet,
    geometry=gpd.points_from_xy(df_geoparquet.lon, df_geoparquet.lat, crs="EPSG:4326"),
)

In [ ]:
# Je réordonne les colonnes dans le bon ordre
gdf = gdf[
    [
        "gtfs_id",
        "navitia_id",
        "osm_id",
        "name",
        "description",
        "line_gtfs_ids",
        "line_osm_ids",
        "geometry",
        "other",
    ]
]

In [ ]:
# je sauvegarde au format dataframe
gdf.to_parquet("../data/C2C/depart_topos_stops_isere.parquet", index=False)